In [1]:
import arcgis
import pandas as pd
from IPython.display import Image

In [2]:
gis = arcgis.GIS("home")

In [ ]:
naip_item = gis.content.get("3f8d2d3828f24c00ae279db4af26d566")
naip_item

In [4]:
lyr_naip = naip_item.layers[0]

In [ ]:
map_view = gis.map("Mountain House, CA")
map_view

In [6]:
map_view.content.add(lyr_naip)

In [7]:
# define extent by tuple
in_extent = (-13530675.22, 4545237.67, -13528866.13, 4547533.23) 

# unpack the extent tuple into individual variables
xmin, ymin, xmax, ymax = in_extent

In [ ]:
# create a dictionary for the envelope
extent_dict = {'xmin': xmin,
  'ymin': ymin,
  'xmax': xmax,
  'ymax': ymax,
  'spatialReference': {'wkid': 3857}}

# create an envelope geometry
envelope = arcgis.geometry.Envelope(extent_dict) 
envelope

In [9]:
# create a polygon from the envelope
p_dict = {'rings': [[
    [xmin, ymin],
    [xmin, ymax],
    [xmax, ymax],
    [xmax, ymin],
    [xmin, ymin]
]], 'spatialReference': {'wkid': 3857}}

poly = arcgis.geometry.Polygon(p_dict)

In [10]:
# create a DataFrame and add to map
df = pd.DataFrame([{"SHAPE": poly}])
df.spatial.set_geometry("SHAPE")
map_view.content.add(df)

In [ ]:
# NAIP Resolution
naip_res = 1 

# Web Mercator meters per unit
metersPerUnit = 1

# calculate the width and height of our output in pixels
w = int((xmax - xmin) * metersPerUnit / naip_res)
h = int((ymax - ymin) * metersPerUnit / naip_res)

[w,h]

In [ ]:
df_naip = lyr_naip.query(
    where = "AcquisitionDate IS NOT NULL",
    geometry_filter = arcgis.geometry.filters.intersects(
        geometry=envelope, 
        sr=3857),
    as_df=True,
    )
df_naip

In [ ]:
df_naip[['Year','AcquisitionDate']]

In [ ]:
filter_date = df_naip['AcquisitionDate'][0]
filter_date

In [ ]:
lyr_naip.export_image?

In [ ]:
mountain_house_2012 = lyr_naip.export_image(
        bbox = envelope, # bounding box generated earlier
        bbox_sr = 3857, # bounding box spatial reference (web mercator)
        image_sr = 3857, # bounding box spatial reference (web mercator)
        time = filter_date, # timestamp representing 2012 acquisition date
        rendering_rule = {'rasterFunction': "NaturalColor"}, # which bands to include
        f = "image", # specify image output
        export_format='jpeg', # output image format
        size = [w,h], # width and height that we generated earlier
        save_folder = r".", # output folder - this is the one that the script is in
        save_file = rf"mountain_house_2012.jpg", # file name
        )

Image(mountain_house_2012)

In [ ]:
years_and_dates = df_naip[['Year','AcquisitionDate']].to_dict('records')
years_and_dates[0]

In [ ]:
for year_and_date in years_and_dates:
    
    # unpack the year
    year = year_and_date['Year']
    
    # unpack the filter date
    filter_date = year_and_date['AcquisitionDate']
    
    # export the image
    export_image = lyr_naip.export_image(
        bbox = envelope,
        bbox_sr = 3857, 
        image_sr = 3857, 
        time = filter_date, # timestamp changed to iterating date
        rendering_rule = {'rasterFunction': "NaturalColor"}, 
        f = "image", 
        export_format='jpeg', 
        size = [w,h], 
        save_folder = r".", 
        save_file = rf"mountain_house_{year}.jpg", # file name changed to include year
        )
    
    print(year)
    display(Image(export_image))

Copyright 2025 Esri